https://observablehq.com/@fjlopez7/decision-tree-vs-random-forest Link a observable

In [0]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.tree.export import export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import json

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [63]:
!ls "/content/drive/Shared drives/Infovis"

 e2_predict.csv		'msd_genre_dataset.zip (Unzipped Files)'
 e2_train.csv		 msd.txt
'Entrega 01.gdoc'	 shortened_dataset.csv
 games-features.csv	 stats_decision_tree.csv
 msd_genre_dataset.csv	 stats_decision_tree.json
 msd_genre_dataset.txt	 stats_random_forest.json
 msd_genre_dataset.zip	 training.ipynb


In [15]:
 '''with open("/content/drive/Shared drives/Infovis/msd_genre_dataset.txt", "r") as file:
  for i in range(9):
    file.readline()
  with open("/content/drive/Shared drives/Infovis/msd_genre_dataset.csv", "w") as file1:
    names_str = file.readline().strip()[1:]
    file1.write(names_str+"\n")
    a = file.readline().strip() + "\n"
    while a != "\n":
      file1.write(a)
      a = file.readline().strip() + "\n"
'''

'with open("/content/drive/Shared drives/Infovis/msd_genre_dataset.txt", "r") as file:\n for i in range(9):\n   file.readline()\n with open("/content/drive/Shared drives/Infovis/msd_genre_dataset.csv", "w") as file1:\n   names_str = file.readline().strip()[1:]\n   file1.write(names_str+"\n")\n   a = file.readline().strip() + "\n"\n   while a != "\n":\n     file1.write(a)\n     a = file.readline().strip() + "\n"\n'

In [64]:

data = pd.read_csv("/content/drive/Shared drives/Infovis/shortened_dataset.csv")
genres = [i for i in pd.factorize(data.genre)[1]]
data.genre = pd.Categorical(pd.factorize(data.genre)[0] + 1)
data.track_id = pd.Categorical(pd.factorize(data.track_id)[0] + 1)
data.artist_name = pd.Categorical(pd.factorize(data.artist_name)[0] + 1)
data.title = pd.Categorical(pd.factorize(data.title)[0] + 1)
data["track_id"] = data['track_id'].astype('int64')
data["genre"] = data['genre'].astype('int64')
data["artist_name"] = data['artist_name'].astype('int64')
data["title"] = data['title'].astype('int64')
num_cols = data._get_numeric_data().columns
print(num_cols)
print(len(data))

Index(['genre', 'track_id', 'artist_name', 'title', 'loudness', 'tempo',
       'time_signature', 'key', 'mode', 'duration', 'avg_timbre1',
       'avg_timbre2', 'avg_timbre3', 'avg_timbre4', 'avg_timbre5',
       'avg_timbre6', 'avg_timbre7', 'avg_timbre8', 'avg_timbre9',
       'avg_timbre10', 'avg_timbre11', 'avg_timbre12', 'var_timbre1',
       'var_timbre2', 'var_timbre3', 'var_timbre4', 'var_timbre5',
       'var_timbre6', 'var_timbre7', 'var_timbre8', 'var_timbre9',
       'var_timbre10', 'var_timbre11', 'var_timbre12'],
      dtype='object')
1000


In [0]:
cant_columns = 5 ## cantidad de columnas a ocupar
X = data.drop('genre', axis=1)
y = data["genre"]
X = X.drop("track_id", axis = 1)
X = X.drop("artist_name", axis = 1)
X = X.drop("title", axis = 1)
## Se sacan todos las columnas que son categoricas (all observar la toma de decisiones 
## solo se basa en estas si es que esta en el modelo )
bestfeatures = SelectKBest(f_classif, k = cant_columns)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score'] 
features = [i for i in featureScores.nlargest(cant_columns,'Score').Specs]

In [0]:
X = data[features]
y = data["genre"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [67]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [68]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
r = export_text(classifier, feature_names=features)
print(r)

0.31
|--- avg_timbre2 <= 5.41
|   |--- loudness <= -16.56
|   |   |--- duration <= 359.46
|   |   |   |--- avg_timbre5 <= 25.61
|   |   |   |   |--- avg_timbre2 <= -153.92
|   |   |   |   |   |--- loudness <= -24.87
|   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |--- loudness >  -24.87
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- avg_timbre2 >  -153.92
|   |   |   |   |   |--- avg_timbre1 <= 36.44
|   |   |   |   |   |   |--- duration <= 104.86
|   |   |   |   |   |   |   |--- duration <= 80.90
|   |   |   |   |   |   |   |   |--- loudness <= -18.18
|   |   |   |   |   |   |   |   |   |--- duration <= 38.71
|   |   |   |   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |   |   |   |   |--- duration >  38.71
|   |   |   |   |   |   |   |   |   |   |--- class: 6
|   |   |   |   |   |   |   |   |--- loudness >  -18.18
|   |   |   |   |   |   |   |   |   |--- duration <= 32.76
|   |   |   |   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |   

In [70]:
import csv

report_decision = classification_report(y_test, y_pred, output_dict=True)
print(report_decision)
f = csv.writer(open("/content/drive/Shared drives/Infovis/stats_decision_tree.csv", "w", newline=''))

# Write CSV Header, If you dont need that, remove this line
f.writerow(["genre", "precision", "recall", "f1-score", "numberclassified"])

for x in report_decision:
    if x.isdigit():
      f.writerow([genres[int(x)-1],
                report_decision[x]["precision"],
                report_decision[x]["recall"],
                report_decision[x]["f1-score"],
                report_decision[x]["support"]])

{'1': {'precision': 0.17647058823529413, 'recall': 0.125, 'f1-score': 0.14634146341463414, 'support': 24}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}, '3': {'precision': 0.25, 'recall': 0.3333333333333333, 'f1-score': 0.28571428571428575, 'support': 9}, '4': {'precision': 0.45454545454545453, 'recall': 0.5555555555555556, 'f1-score': 0.5, 'support': 9}, '5': {'precision': 0.47058823529411764, 'recall': 0.45714285714285713, 'f1-score': 0.46376811594202894, 'support': 70}, '6': {'precision': 0.2, 'recall': 0.25, 'f1-score': 0.22222222222222224, 'support': 4}, '7': {'precision': 0.4594594594594595, 'recall': 0.3333333333333333, 'f1-score': 0.38636363636363635, 'support': 51}, '8': {'precision': 0.038461538461538464, 'recall': 0.06666666666666667, 'f1-score': 0.04878048780487805, 'support': 15}, '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7}, '10': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'accuracy': 0.31, 'macro av

In [0]:
report_decision = classification_report(y_test, y_pred, output_dict=True)
to_json_1 = dict()
for i in report_decision:
  if i.isdigit():
    to_json_1[genres[int(i)-1]] = report_decision[i]
#with open("stad_tree.csv", "w") as file:
to_json_1["accuracy"] = accuracy_score(y_test,y_pred)  
with open("/content/drive/Shared drives/Infovis/stats_decision_tree.json","w") as file:
  json.dump(to_json_1, file)

In [72]:
random_cls = RandomForestClassifier(n_estimators=100)
random_cls.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [73]:
y_pred_r = random_cls.predict(X_test)
print(classification_report(y_test, y_pred_r))

              precision    recall  f1-score   support

           1       0.45      0.21      0.29        24
           2       0.14      0.11      0.12         9
           3       0.60      0.33      0.43         9
           4       0.67      0.44      0.53         9
           5       0.42      0.77      0.54        70
           6       0.25      0.25      0.25         4
           7       0.52      0.31      0.39        51
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         7
          10       0.00      0.00      0.00         2

    accuracy                           0.42       200
   macro avg       0.30      0.24      0.26       200
weighted avg       0.40      0.42      0.38       200



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
print(classifier.decision_path(X_test))

  (0, 0)	1
  (0, 440)	1
  (0, 540)	1
  (0, 541)	1
  (0, 542)	1
  (0, 543)	1
  (0, 545)	1
  (0, 546)	1
  (0, 547)	1
  (0, 555)	1
  (0, 567)	1
  (0, 568)	1
  (0, 569)	1
  (1, 0)	1
  (1, 440)	1
  (1, 540)	1
  (1, 541)	1
  (1, 542)	1
  (1, 543)	1
  (1, 545)	1
  (1, 579)	1
  (1, 589)	1
  (2, 0)	1
  (2, 440)	1
  (2, 540)	1
  :	:
  (197, 730)	1
  (197, 731)	1
  (198, 0)	1
  (198, 1)	1
  (198, 101)	1
  (198, 125)	1
  (198, 126)	1
  (198, 127)	1
  (198, 128)	1
  (198, 129)	1
  (198, 147)	1
  (198, 153)	1
  (198, 154)	1
  (198, 155)	1
  (198, 161)	1
  (199, 0)	1
  (199, 440)	1
  (199, 441)	1
  (199, 442)	1
  (199, 443)	1
  (199, 455)	1
  (199, 457)	1
  (199, 485)	1
  (199, 491)	1
  (199, 493)	1


In [75]:
report_decision_forest = classification_report(y_test, y_pred_r, output_dict=True)
to_json_2 = dict()
for i in report_decision_forest:
  if i.isdigit():
    to_json_2[genres[int(i)-1]] = report_decision_forest[i]
#with open("stad_tree.csv", "w") as file:
to_json_2["accuracy"] = accuracy_score(y_test,y_pred_r)  
with open("/content/drive/Shared drives/Infovis/stats_random_forest.json","w") as file:
  json.dump(to_json_2, file)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [77]:

print(report_decision_forest)
f = csv.writer(open("/content/drive/Shared drives/Infovis/stats_decision_forest.csv", "w", newline=''))

# Write CSV Header, If you dont need that, remove this line
f.writerow(["genre", "precision", "recall", "f1-score", "numberclassified"])

for x in report_decision_forest:
    if x.isdigit():
      f.writerow([genres[int(x)-1],
                report_decision_forest[x]["precision"],
                report_decision_forest[x]["recall"],
                report_decision_forest[x]["f1-score"],
                report_decision_forest[x]["support"]])

{'1': {'precision': 0.45454545454545453, 'recall': 0.20833333333333334, 'f1-score': 0.28571428571428575, 'support': 24}, '2': {'precision': 0.14285714285714285, 'recall': 0.1111111111111111, 'f1-score': 0.125, 'support': 9}, '3': {'precision': 0.6, 'recall': 0.3333333333333333, 'f1-score': 0.42857142857142855, 'support': 9}, '4': {'precision': 0.6666666666666666, 'recall': 0.4444444444444444, 'f1-score': 0.5333333333333333, 'support': 9}, '5': {'precision': 0.4186046511627907, 'recall': 0.7714285714285715, 'f1-score': 0.542713567839196, 'support': 70}, '6': {'precision': 0.25, 'recall': 0.25, 'f1-score': 0.25, 'support': 4}, '7': {'precision': 0.5161290322580645, 'recall': 0.3137254901960784, 'f1-score': 0.3902439024390244, 'support': 51}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 15}, '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7}, '10': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'accuracy': 0.42, 'macro avg': {'pr